### Proyecto 1 - Procesamiento de datos utilizando Apache Hive y Apache Zeppelin

#### Análisis de Olimpiadas

![Olympic Logo](https://pyt-blogs.imgix.net/2016/08/20140204000881966198-original.jpg)

#### Paso 1 - Descarga de archivos de datos que vamos a procesar

In [ ]:
# Creamos los directorios
hdfs dfs -mkdir -p /home/dataengineer/files
hdfs dfs -mkdir -p /user/dataengineer/filesdata

# Entrar al directorio
cd /home/dataengineer/files

# Descargamos de la red los archivos que vamos a utilizar
wget https://raw.githubusercontent.com/hivesample/sample/main/athlete_events.csv
wget https://raw.githubusercontent.com/hivesample/sample/main/noc_regions.csv 

# Luego copiamos los archivos a HDFS
hdfs dfs -copyFromLocal athlete_events.csv /user/dataengineer/filesdata
hdfs dfs -copyFromLocal noc_regions.csv /user/dataengineer/filesdata

#### Paso 2 - Descarga y ejecución de Apache Zeppelin

In [ ]:
# Crear un directorio
hdfs dfs -mkdir /home/dataengineer/zeppelin

# Entrar al directorio
cd /home/dataengineer/zeppelin

# Descargar Apache Zeppelin
wget https://dlcdn.apache.org/zeppelin/zeppelin-0.10.1/zeppelin-0.10.1-bin-all.tgz

# Una vez Descargado
tar -xvzf zeppelin-0.10.1-bin-all.tgz

# Iniciar Apache Zeppelin usando comando
bin/zeppelin-daemon.sh start

# Detener Apache Zeppelin usando comando
bin/zeppelin-daemon.sh stop

#### Paso 3 - Configuración del intérprete Hive en la interfaz gráfica de Apache Zeppelin

In [ ]:
# Desde el navegador ingresamos a la interfaz gráfica de zeppelin
https://localhost:8080/

# Damos clic en "anonymous" y luego seleccionamos "Interpreter"

# Luego, creamos un nuevo interprete:
# Interpreter name: hive
# Interpreter group: jdbc

# En "Propiedades" ingresamos los siguientes datos:
# default.url: jdbc:hive2://localhost:10000
# default.user: dataengineer
# default.password: (queda en blanco)
# default.driver: org.apache.hive.jdbc.HiveDriver

# Todas las otras propiedades que vienen a continuación de "default.driver" las eliminamos. 
# Solo necesitamos mantener 4 propiedades

# Todo esto se extrajo de la documentación de Apache Zeppelin
https://zeppelin.apache.org/docs/0.10.1/interpreter/hive.html

# Para las "Dependencias" ingresamos lo siguiente:
# org.apache.hive:hive-jdbc:0.14.0
# org.apache.hadoop:hadoop-common:2.6.0	

# Finalmente "Guardamos"

#### Paso 4 - Iniciar Hadoop, Hive y Zeppelin

In [ ]:
# Buscamos el directorio donde realizamos la instalación de hadoop e ingresamos
cd /home/dataengineer/hadoop/hadoop-3.3.2/

# Iniciamos el sistema de archivos distribuidos
sbin/start-dfs.sh

# Iniciamos yarn
sbin/start-yarn.sh

# Buscamos el directorio donde realizamos la instalacion de hive e ingresamos
cd /home/dataengineer/apachehive/apache-hive-3.1.2-bin/

# Ejecutamos hive
bin/hiveserver2

# Buscamos el directorio donde realizamos la instalacion de zeppelin e ingresamos
cd /home/dataengineer/zeppelin/zeppelin-0.10.1-bin-all/ 

# Ejecutamos zeppelin
bin/zeppelin-daemon.sh start

# Desde el navegador ingresamos a la interfaz gráfica de zeppelin
https://localhost:8080/

#### Paso 5 - Creación de tablas y carga de datos en el notebook Zepellin

In [ ]:
# Creamos la tabla athlete_events_final
# Recordar que 'separatorChar' nos indica el separador 
# y 'quoteChar' nos indica el caracter a ignorar que se encuentre entre los separadores

%Hive

CREATE TABLE IF NOT EXISTS athlete_events_final (
ID INT,
Name string, 
Sex string, 
Age INT, 
Height INT, 
Weight INT,
Team string, 
NOC string, 
Games string, 
Year INT, 
Season string, 
City string, 
Sport string, 
Event string, 
Medal string)
COMMENT 'athlete events table'
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES 
(
    "separatorChar" = ",",
    "quoteChar"     = "\""
)  
TBLPROPERTIES("skip.header.line.COUNT"="1") 

In [ ]:
# Carga de datos

%Hive

LOAD DATA INPATH '/user/dataengineer/filesdata/athlete_events.csv' INTO TABLE athlete_events_final;

In [ ]:
# Lanzamos una consulta para verificar que los datos se hayan cargado correctamente

%Hive

SELECT * FROM athlete_events_final;

In [ ]:
# Creamos la tabla noc_regions

%Hive

CREATE TABLE IF NOT EXISTS noc_regions (
NOC string,
region string,
notes string)
COMMENT 'noc regions table'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
TBLPROPERTIES("skip.header.line.COUNT"="1"); 


In [ ]:
# Carga de datos

%Hive

LOAD DATA INPATH '/user/dataengineer/filesdata/noc_regions.csv' INTO TABLE noc_regions;

In [ ]:
# Lanzamos una consulta para verificar que los datos se hayan cargado correctamente

%Hive

SELECT * FROM noc_regions;

#### Paso 6 - Análisis de datos

##### Distribución de edad para los deportistas con medalla de oro

In [ ]:
%Hive

SELECT COUNT(Medal) AS Medals, Age 
FROM athlete_events_final 
WHERE Medal = 'Gold' 
GROUP BY Age 
ORDER BY Age ASC;

##### Medallas de oro para deportistas de más de 50 años según deportes

In [ ]:
%Hive

SELECT Sport, Age 
FROM athlete_events_final 
WHERE Medal = '"Gold"' 
AND Age >= 50; 

##### Medallas femeninas por edición (temporada de verano) de los juegos

In [ ]:
%Hive

SELECT COUNT(Medal) AS Medals, Year 
FROM athlete_events_final 
WHERE Sex = '"F"' 
AND Season = '"Summer"' 
AND Medal IN ('"Bronze"','"Gold"','"Silver"') 
GROUP BY Year ORDER BY Year ASC;

##### 5 países con más medallas de oro

In [ ]:
%Hive

SELECT COUNT(Medal) AS Medals, region 
FROM athlete_events_final A 
JOIN noc_regions N 
ON A.NOC = N.NOC  
WHERE Medal = 'Gold' 
GROUP BY region 
ORDER BY Medals DESC LIMIT 5; 

##### Disciplinas con mayor número de medallas de oro para USA

In [ ]:
%Hive

SELECT COUNT(Medal) AS Medals, Event 
FROM athlete_events_final A 
JOIN noc_regions N 
ON A.NOC = N.NOC  
WHERE Medal = 'Gold' 
AND A.NOC = 'USA' 
GROUP BY Event 
ORDER BY Medals DESC; 

##### Altura frente a Peso de los medallistas de oro olímpicos

In [ ]:
%Hive

SELECT Weight, Height 
FROM athlete_events_final 
WHERE  Medal = 'Gold'; 

##### Variación del atleta masculino a lo largo del tiempo

In [ ]:
%Hive

SELECT COUNT(Sex) AS Males, Year 
FROM athlete_events_final 
WHERE Sex = 'M' 
AND Season = 'Summer'
GROUP BY Year
ORDER BY Year ASC; 

##### Variación del atleta femenino a lo largo del tiempo

In [ ]:
%Hive

SELECT COUNT(Sex) AS Females, Year 
FROM athlete_events_final 
WHERE Sex = 'F' 
AND Season = 'Summer'
GROUP BY Year 
ORDER BY Year ASC; 

##### Variación de la edad de los deportistas masculinos a lo largo del tiempo

In [ ]:
%Hive

SELECT MIN(Age), AVG(Age), MAX(Age), Year 
FROM athlete_events_final 
WHERE Sex = 'M' 
GROUP BY Year 
ORDER BY Year ASC; 

##### Variación de la edad de las deportistas femeninas a lo largo del tiempo

In [ ]:
%Hive

SET hive.auto.convert.join=false;

SELECT MIN(Age), AVG(Age), MAX(Age), Year 
FROM athlete_events_final 
WHERE Sex = 'F' 
GROUP BY Year 
ORDER BY Year ASC; 

##### Variación del peso de los deportistas masculinos a lo largo del tiempo

In [ ]:
%Hive

SELECT MIN(Weight), AVG(Weight), MAX(Weight), Year 
FROM athlete_events_final 
WHERE Sex = 'M' 
GROUP BY Year 
ORDER BY Year ASC; 

##### Variación del peso de las deportistas femeninas a lo largo del tiempo

In [ ]:
%Hive

SELECT MIN(Weight), AVG(Weight), MAX(Weight), Year 
FROM athlete_events_final 
WHERE Sex = 'F' 
AND Year > 1925 
GROUP BY Year 
ORDER BY Year ASC; 

##### Variación de la altura de los deportistas masculinos a lo largo del tiempo

In [ ]:
%Hive

SELECT MIN(Height), AVG(Height), MAX(Height), Year 
FROM athlete_events_final 
WHERE Sex = 'M' 
GROUP BY Year 
ORDER BY Year ASC; 

##### Variación de la altura de las deportistas femeninas a lo largo del tiempo

In [ ]:
%Hive

SELECT MIN(Height), AVG(Height), MAX(Height), Year 
FROM athlete_events_final 
WHERE Sex = 'F' 
GROUP BY Year 
ORDER BY Year ASC; 

##### Peso a lo largo de los años de los gimnastas masculinos

In [ ]:
%Hive

SELECT MIN(Weight), AVG(Weight), MAX(Weight), Year 
FROM athlete_events_final 
WHERE Sport = 'Gymnastics' 
AND Sex = 'M' 
AND Year > 1950 
GROUP BY Year 
ORDER BY Year;

##### Peso a lo largo de los años de las gimnastas femeninas

In [ ]:
%Hive

SELECT MIN(Weight), AVG(Weight), MAX(Weight), Year 
FROM athlete_events_final 
WHERE Sport = 'Gymnastics' 
AND Sex = 'F' 
AND Year > 1950 
GROUP BY Year ORDER BY Year;

##### Peso a lo largo de los años para levantadores masculinos

In [ ]:
%Hive

SELECT MIN(Weight), AVG(Weight), MAX(Weight), Year 
FROM athlete_events_final 
WHERE Sport = 'Weightlifting' 
AND Sex = 'M' 
AND Year > 1950 
GROUP BY Year ORDER BY Year;

##### Peso a lo largo de los años para levantadoras femeninos

In [ ]:
%Hive

SELECT MIN(Weight), AVG(Weight), MAX(Weight), Year 
FROM athlete_events_final 
WHERE Sport = 'Weightlifting' 
AND Sex = 'F' 
AND Year > 1950 
GROUP BY Year 
ORDER BY Year;

##### Altura a lo largo de los años para levantadores masculinos

In [ ]:
%Hive

SELECT MIN(Height), AVG(Height), MAX(Height), Year 
FROM athlete_events_final 
WHERE Sport = 'Weightlifting' 
AND Sex = 'M' 
AND Year > 1950 
GROUP BY Year 
ORDER BY Year;

##### Altura a lo largo de los años para levantadoras femeninas

In [ ]:
%Hive

SELECT MIN(Height), AVG(Height), MAX(Height), Year 
FROM athlete_events_final 
WHERE Sport = 'Weightlifting' 
AND Sex = 'F' 
AND Year > 1950 
GROUP BY Year 
ORDER BY Year;

##### Medallas de oro según pais

In [ ]:
%Hive

SELECT COUNT(Medal) AS Medals, N.NOC 
FROM athlete_events_final A 
JOIN noc_regions N 
ON A.NOC = N.NOC  
WHERE Medal = 'Gold' 
GROUP BY N.NOC

##### Medallas de plata según pais

In [ ]:
%Hive

SELECT COUNT(Medal) AS Medals, N.NOC 
FROM athlete_events_final A 
JOIN noc_regions N 
ON A.NOC = N.NOC  
WHERE Medal = 'Silver' 
GROUP BY N.NOC

##### Medallas de bronce según pais

In [ ]:
%Hive

SELECT COUNT(Medal) AS Medals, N.NOC 
FROM athlete_events_final A 
JOIN noc_regions N 
ON A.NOC = N.NOC  
WHERE Medal = 'Bronze' 
GROUP BY N.NOC